1. Download

In [ ]:
#pip install opencv-python matplotlib tensorflow mediapipe scikit-learn

2. Import

In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt    # MIGHT REMOVE
import time                             # MIGHT REMOVE
import mediapipe as mp
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard # MIGHT REMOVE

mp_pose = mp.solutions.pose
mp_draw = mp.solutions.drawing_utils

3. Functions

In [ ]:
# Mediapipe Detection Function
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Color Conversion BGR to RGB
    image.flags.writeable = False                   # Non-writeable
    results = model.process(image)                  # Make Prediction
    image.flags.writeable = True                    # Writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # Color Conversion RGB back to BGR
    return image, results

In [ ]:
# Mediapipe Draw Function
def draw_styled_landmarks(image, results):
    mp_draw.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,                     # Draw Face
                              mp_draw.DrawingSpec(color=(255,255,255), thickness=1, circle_radius=1),   # Face Dot Color
                              mp_draw.DrawingSpec(color=(240,19,255), thickness=1, circle_radius=1),    # Face Line Color
                              )

In [ ]:
# Extract Keypoints Function
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*3)
    return pose

TEST Pose Detection

In [ ]:
cap = cv2.VideoCapture(0)
# Set Mediapipe Model
with mp_pose.Pose(min_detection_confidence=0.8, min_tracking_confidence=0.8) as pose_model:
    while cap.isOpened():
        
        # Read Feed
        ret, frame = cap.read()
        
        # Make Detections
        image, results = mediapipe_detection(frame, pose_model)
        print(results)
        
        # Draw Landmarks
        draw_styled_landmarks(image, results)

        # Show Feed
        cv2.imshow('OpenCV Feed', image)

        # Break Loop
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    # If Broken
    cap.release()
    cv2.destroyAllWindows()

4. Training

In [ ]:
# Path for stored data
DATA_PATH = os.path.join('MP_DATA30n+')

# Actions to detect
#actions = np.array(['deadlift-s', 'deadlift-c', 'squat-s', 'squat-c']) # S = Starting Point, C = Contraction Point
actions = np.array(['nothing', 'deadlift-s', 'deadlift-c', 'squat-s', 'squat-c']) # S = Starting Point, C = Contraction Point

sequence_length = 30

In [ ]:
# Create Directory
for action in actions: 
    os.makedirs(os.path.join(DATA_PATH, action))

In [ ]:
cap = cv2.VideoCapture(0)
tempPath = 'ExtraData'

with mp_pose.Pose(min_detection_confidence=0.8, min_tracking_confidence=0.8) as pose_model:
    while cap.isOpened():
        
        ret, frame = cap.read()
        image, results = mediapipe_detection(frame, pose_model)
        draw_styled_landmarks(image, results)

        cv2.putText(image, 'Collectiong frames for {}'.format(action), (0,12),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)

        cv2.imshow('Data Capture', image)

        key = cv2.waitKey(10) & 0xFF
        if key == ord('1'):
            action = 'nothing'
        elif key == ord('2'):
            if action == actions[1]:
                action = actions[2]
            else:
                action = actions[1]
        elif key == ord('3'):
            if action == actions[3]:
                action = actions[4]
            else:
                action = actions[3]
        elif key == ord(' '):
            # Make New Directory
            name = str(len(os.listdir(os.path.join(tempPath, action))))
            os.makedirs(os.path.join(tempPath, action, name))

            for i in range(sequence_length):
                ret, frame = cap.read()
                image, results = mediapipe_detection(frame, pose_model)
                print(results)
                draw_styled_landmarks(image, results)
                cv2.imshow('Data Capture', image)
                npy_path = os.path.join(tempPath, action, name, str(i))
                keypoints = extract_keypoints(results)
                np.save(npy_path, keypoints)
        elif key == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [ ]:
cap.release()
cv2.destroyAllWindows()

In [ ]:
label_map = {label:num for num, label in enumerate(actions)}

In [ ]:
label_map

In [ ]:
no_sequences = 30 # Number of recordings made for each action

sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [ ]:
no_sequences = 30 # Number of recordings made for each action

sequences, labels = [], []
for action in actions:
    if action == 'nothing':
        no_sequences = 45
    else:
        no_sequences = 30
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [ ]:
labels

In [ ]:
sequences

In [ ]:
x = np.array(sequences)

In [ ]:
y = to_categorical(labels).astype(int)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.05)

In [ ]:
log_dir = os.path.join('logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,99)))    # 10/30 = frames
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy']) # Look up optimizer

In [ ]:
model.fit(x_train, y_train, epochs=70, callbacks=[tb_callback]) # Be vary of the epochs, potentially stop early

In [ ]:
del model

In [ ]:
model.save('exercise30n+.keras')
model.save('exercise30n+.h5')

In [ ]:
model = tf.keras.models.load_model('Models\\exercise30n+.keras')

5. Test Model

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [ ]:
# Look in other project when ready to test model

In [ ]:
testRes = model.predict(x_test)

In [ ]:
testRes

In [94]:
np.sum(testRes[3])

1.0

In [95]:
actions[np.argmax((testRes[3]))]

'deadlift-c'

In [96]:
actions[np.argmax(y_test[3])]

'deadlift-s'

In [100]:
yhat = model.predict(x_train)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 


In [101]:
ytrue = np.argmax(y_train, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [104]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[114,   0],
        [  4,  38]],

       [[127,   1],
        [  0,  28]],

       [[127,   0],
        [  0,  29]],

       [[126,   3],
        [  1,  26]],

       [[125,   1],
        [  0,  30]]], dtype=int64)

6. Live Predictions

In [ ]:
sequence = []
res = [0]

cap = cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.8, min_tracking_confidence=0.8) as pose_model:
    while cap.isOpened():
        ret, frame = cap.read()
        image, results = mediapipe_detection(frame, pose_model)
        draw_styled_landmarks(image, results)

        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        currentFrame = len(sequence)

        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            sequence = sequence[-20:] # Delay

        currentConf =  res[np.argmax(res)]
        currentRes = actions[np.argmax(res)]

        if currentConf >= 0.7:
            cv2.putText(image, 'Prediction: {}'.format(currentRes), (0,34),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,0,0), 2, cv2.LINE_AA)
            cv2.putText(image, 'Confidence: {}'.format(currentConf), (0,50),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, 'Frames: {}'.format(currentFrame), (0,70), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        elif currentConf >= 0.5:
            cv2.putText(image, 'Prediction: {}'.format(currentRes), (0,34),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,165,255), 2, cv2.LINE_AA)
            cv2.putText(image, 'Confidence: {}'.format(currentConf), (0,50),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,165,255), 1, cv2.LINE_AA)
            cv2.putText(image, 'Frames: {}'.format(currentFrame), (0,70), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,165,255), 1, cv2.LINE_AA)
        else:
            cv2.putText(image, 'Prediction: {}'.format(currentRes), (0,34),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,0,255), 2, cv2.LINE_AA)
            cv2.putText(image, 'Confidence: {}'.format(currentConf), (0,50),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)
            cv2.putText(image, 'Frames: {}'.format(currentFrame), (0,70), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)

        cv2.imshow('Prediction Feed', image)

        key = cv2.waitKey(10) & 0xFF
        if key == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [ ]:
cap.release()
cv2.destroyAllWindows()